In [1]:
from rdkit import Chem
from rdmc import RDKitMol

from rdmc.conformer_generation.ts_generators import TSConformerGenerator
from rdmc.conformer_generation.ts_guessers import TSEGNNGuesser
from rdmc.conformer_generation.ts_optimizers import SellaOptimizer

from rdmc.view import interactive_conformer_viewer
from rdmc.external.xtb_tools.opt import run_xtb_calc

In [14]:
# specify reaction

# rxn_smiles = '[C:11]1([H:23])([H:24])[C:12]([H:25])([H:26])[C:13]([H:27])([H:28])[C:14]([H:29])=[C:15]([H:30])[C:16]1([H:31])[H:32].[c:1]1([H:17])[c:2]([H:18])[c:3]([H:19])[c:4]([C:7](=[O:8])[O:9][O:10][H:22])[c:5]([H:20])[c:6]1[H:21]>>[C:11]1([H:23])([H:24])[C:12]([H:25])([H:26])[C:13]([H:27])([H:28])[C:14]2([H:29])[O:10][C:15]2([H:30])[C:16]1([H:31])[H:32].[c:1]1([H:17])[c:2]([H:18])[c:3]([H:19])[c:4]([C:7]([O:8][H:22])=[O:9])[c:5]([H:20])[c:6]1[H:21]'
# rxn_smiles = '[C:1]([C:2]([C:3]([H:20])([H:21])[H:22])([O:4])[C:5]([H:23])([H:24])[H:25])([H:17])([H:18])[H:19].[C:6]([C:7]([C:8]([H:29])([H:30])[H:31])([C:9]([H:32])([H:33])[H:34])[c:10]1[c:11]([H:35])[c:12]([H:36])[c:13]([O:14][H:37])[c:15]([H:38])[c:16]1[H:39])([H:26])([H:27])[H:28]>>[C:1]([C:2]([C:3]([H:20])([H:21])[H:22])([O:4][H:37])[C:5]([H:23])([H:24])[H:25])([H:17])([H:18])[H:19].[C:6]([C:7]([C:8]([H:29])([H:30])[H:31])([C:9]([H:32])([H:33])[H:34])[c:10]1[c:11]([H:35])[c:12]([H:36])[c:13]([O:14])[c:15]([H:38])[c:16]1[H:39])([H:26])([H:27])[H:28]'
# rxn_smiles = "[C:1]([C@@:2]([O:3][H:12])([C:4]([N:5]([C:6](=[O:7])[H:16])[H:15])([H:13])[H:14])[H:11])([H:8])([H:9])[H:10]>>[C:1]([C@@:2]([O:3][H:12])([C:4](/[N:5]=[C:6](/[O:7][H:15])[H:16])([H:13])[H:14])[H:11])([H:8])([H:9])[H:10]"

# rxn_smiles = '[C:1](=[C:2]([C:3]([C:4]([C:5](=[C:6]([C:7]([H:17])[H:18])[H:16])[H:15])([H:13])[H:14])([H:11])[H:12])[H:10])([H:8])[H:9]>>[C:1]([C:2]1([H:10])[C:3]([H:11])([H:12])[C:4]([H:13])([H:14])[C:5]([H:15])=[C:6]([H:16])[C:7]1([H:17])[H:18])([H:8])[H:9]'
rxn_smiles = '[C:1](=[C:2]([C:3]([C:4]([C:5]([C:6](=[C:7]([H:17])[H:18])[H:16])[H:15])([H:13])[H:14])([H:11])[H:12])[H:10])([H:8])[H:9]>>[C:1]1([H:8])([H:9])[C:2]([H:10])[C:3]([H:11])([H:12])[C:4]([H:13])([H:14])[C@@:5]1([C:6](=[C:7]([H:17])[H:18])[H:16])[H:15]'
# rxn_smiles = '[N:1]([c:2]1[c:3]([H:9])[n:4]([H:10])[c:5]([H:11])[n:6]1)([H:7])[H:8]>>[N:1]([C:2]1=[N:6][C:5]([H:11])=[N:4][C:3]1([H:9])[H:10])([H:7])[H:8]'
# rxn_smiles = "[C:1]([C:2]([C:3]([H:11])([H:12])[H:13])([C:4]([C:5]#[N:6])([H:14])[H:15])[H:10])([H:7])([H:8])[H:9]>>[C:1]([C:2]([C:3]([H:11])([H:12])[H:13])([C:4]([N+:6]#[C-:5])([H:14])[H:15])[H:10])([H:7])([H:8])[H:9]"
# rxn_smiles = "[C:1]([C:2]([C:3]([H:11])([H:12])[H:13])([C:4]([C:5]#[N:6])([H:14])[H:15])[H:10])([H:7])([H:8])[H:9]>>[C:1]([C:2]([C:3]([H:11])([H:12])[H:13])=[C:4]([C:5]#[N:6])[H:14])([H:7])([H:8])[H:9].[H:10][H:15]"

In [15]:
trained_model_dir = "../../ts_egnn/trained_models/2022_02_07/"

ts_gen = TSConformerGenerator(
    rxn_smiles=rxn_smiles,
    embedder=TSEGNNGuesser(trained_model_dir, track_stats=True),
    optimizer=SellaOptimizer(track_stats=True),
    save_dir="./test_ts"
)

In [16]:
opt_ts_mol = ts_gen(20)

2022/03/07 05:28:48 PM | TSConformerGenerator | INFO: Embedding stable species conformers...
2022/03/07 05:28:48 PM | StochasticConformerGenerator | INFO: Config specified: using default settings for loose config
2022/03/07 05:28:48 PM | StochasticConformerGenerator | INFO: Generating conformers for [C:1](=[C:2]([C:3]([C:4]([C:5]([C:6](=[C:7]([H:17])[H:18])[H:16])[H:15])([H:13])[H:14])([H:11])[H:12])[H:10])([H:8])[H:9]
2022/03/07 05:28:48 PM | StochasticConformerGenerator | INFO: 
Iteration 1: embedding 20 initial guesses...
2022/03/07 05:28:48 PM | StochasticConformerGenerator | INFO: Iteration 1: optimizing initial guesses...
2022/03/07 05:28:48 PM | StochasticConformerGenerator | INFO: Iteration 1: pruning conformers...
2022/03/07 05:28:48 PM | StochasticConformerGenerator | INFO: Iteration 1: kept 13 unique conformers
2022/03/07 05:28:48 PM | StochasticConformerGenerator | INFO: 
Iteration 2: embedding 20 initial guesses...
2022/03/07 05:28:48 PM | StochasticConformerGenerator | IN

20 10


2022/03/07 05:28:57 PM | TSConformerGenerator | INFO: Optimizing TS guesses...


In [13]:
interactive_conformer_viewer(opt_ts_mol)

interactive(children=(IntSlider(value=0, description='confId', max=10), Output()), _dom_classes=('widget-inter…

<function rdmc.view.interactive_conformer_viewer.<locals>.<lambda>(confId)>

In [18]:
interactive_conformer_viewer(opt_ts_mol_2)

interactive(children=(IntSlider(value=0, description='confId', max=9), Output()), _dom_classes=('widget-intera…

<function rdmc.view.interactive_conformer_viewer.<locals>.<lambda>(confId)>

In [ ]:
for i in range(opt_ts_mol.GetNumConformers()):
    props = run_xtb_calc(opt_ts_mol, confId=i, job="--hess")
    print(props["frequencies"][:5])